# <span style="color:#2c061f"> Macro 318: Tutorial #2 </span>  

<br>

## <span style="color:#374045"> Optimisation and the consumer problem </span>


#### <span style="color:#374045"> Lecturer: </span> <span style="color:#d89216"> <br> Dawie van Lill (dvanlill@sun.ac.za) </span>

# Introduction

Before getting started with this tutorial I want to acknowledge the excellent work being done by Jeppe Druedahl and his team at Copenhagen. The following tutorial is basically a port of the Python lecture for his course on numerical methods. Much of the material is directly copied, but there are also some smaller amendments to fit the 318 course. The original lecture can be found [here](https://numeconcopenhagen.netlify.app/lectures/Optimize_print_and_plot).

In this second tutorial we will be going over some basic data work and the solution of a basic optimisation problem in macroeconomics. We will report our results via text and figures, so this will entail some visualisation. 

The following packages are going to be needed for this tutorial. 

In [1]:
import Pkg

In [14]:
Pkg.add("DataFrames");
Pkg.add("Optim"); # Optimisation package
Pkg.add("Plots");

   Resolving package versions...
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Project.toml`
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Manifest.toml`
Precompiling project...
  ✓ Plots
  1 dependency successfully precompiled in 36 seconds (160 already precompiled)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
   Resolving package versions...
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Project.toml`
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Manifest.toml`
Precompiling project...
  ✓ Plots
  1 dependency successfully precompiled in 37 seconds (160 already precompiled)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
   Resolving package versions...
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Project.toml`
  No Changes to `~/Dropbox/2022/318-macro/git/Macro-318/Manifest.toml`
Precompiling project...


In [5]:
using DataFrames
using Optim
using Plots

## Consumer problem

Consider the following consumer problem. You should be quite familiar with this, since it is very similar to the one that we covered in chapter 4 of Williamson last year. 

We have an economy with two goods and the following components, 

* utility function $u(x_1,x_2):\mathbb{R}^2_{+}\rightarrow\mathbb{R}$,
* exogenous income $I$, and 
* price-vector $(p_1,p_2)$,

We can write this problem as follows, 


$$
\begin{aligned}
V(p_{1},p_{2},I) & = \max_{x_{1},x_{2}}u(x_{1},x_{2})\\
 \text{s.t.}\\
p_{1}x_{1}+p_{2}x_{2} & \leq I,\,\,\,p_{1},p_{2},I>0\\
x_{1},x_{2} & \geq 0
\end{aligned}
$$

Normally these types of problems can be solved by hand. We will solve this particular problem by hand in the tutorial. If we had, for example, a Cobb-Douglas utility function, such as, 

$$
u(x_1,x_2) = x_1^{\alpha}x_2^{1-\alpha}
$$

then the solution is given by, 

$$
\begin{aligned}
x_1^{\ast} &= \alpha \frac{I}{p_1} \\
x_2^{\ast} &= (1-\alpha) \frac{I}{p_2}
\end{aligned}
$$

which implies that $\alpha$ is the budget share of the first good and $1-\alpha$ is the budget share of the second good. 

While this is not a difficult problem to solve by hand, it would be way more convenient to be able to do this on a computer. One of the main reasons is that if the utility function changes we don't need to do the whole process again, we simply need to alter the utility function in our code and we can calculate the new solution. 

## Utility function

Let us write down the utility function in terms of code. Remember how to construct functions from the first lecture? If not, please consult the section on functions. 

In [16]:
function u_func(x1, x2, α = 0.5)
    return x1 ^ α * x2 ^ (1 - α)
end;

We can evaluate this utility function for different values of the inputs $x_1$ and $x_2$.

In [17]:
x1 = 1
x2 = 3

u = u_func(x1, x2)

println("x1 = $x1, x2 = $x2 -> u = $u")

x1 = 1, x2 = 3 -> u = 1.7320508075688772


We can also loop through multiple values for $x_1$ to see how the values for the utility function changes. This reflects the elasticity of utility with respect to $x_1$

In [19]:
x1_values = [2, 4, 6, 8, 10, 12, 14, 16]
x2 = 3

for x1 in x1_values
    u = u_func(x1, x2)

    println("x1 = $x1, x2 = $x2 -> u = $u")
end

x1 = 2, x2 = 3 -> u = 2.4494897427831783
x1 = 4, x2 = 3 -> u = 3.4641016151377544
x1 = 6, x2 = 3 -> u = 4.242640687119285
x1 = 8, x2 = 3 -> u = 4.898979485566357
x1 = 10, x2 = 3 -> u = 5.477225575051661
x1 = 12, x2 = 3 -> u = 5.999999999999999
x1 = 14, x2 = 3 -> u = 6.4807406984078595
x1 = 16, x2 = 3 -> u = 6.928203230275509


For a little nicer output we can use the `enumerate` command, 

In [20]:
for (i, x1) in enumerate(x1_values)
    u = u_func(x1, x2)

    println("$i: x1 = $x1, x2 = $x2 -> u = $u")
end


1: x1 = 2, x2 = 3 -> u = 2.4494897427831783
2: x1 = 4, x2 = 3 -> u = 3.4641016151377544
3: x1 = 6, x2 = 3 -> u = 4.242640687119285
4: x1 = 8, x2 = 3 -> u = 4.898979485566357
5: x1 = 10, x2 = 3 -> u = 5.477225575051661
6: x1 = 12, x2 = 3 -> u = 5.999999999999999
7: x1 = 14, x2 = 3 -> u = 6.4807406984078595
8: x1 = 16, x2 = 3 -> u = 6.928203230275509


We can now see how utility changes with different values for $x_1$. In this case it seems that there is an increase in utility associated with an increase in $x_1$. 

In mathematical terms, it seems that the utility function is monotonically increasing with respect to changes in $x_1$. The way that we would determine the change in the utility function with respect to $x_1$ over the entire function is by using the tools of calculus. 

## Plotting the utility function

The next thing we might want to do in our analysis is to plot the utility function. We want some idea of what the function looks like graphically. We could draw it by hand, but it is much easier to utilise the computer. 

The first thing we will do is to create a 2-dimensional grid with $N$ elements in each dimension. We are going to be plotting our utility function with respect to two inputs, so we need a 2-dimensional grid. 

In [34]:
N = 100 # number of elements
x_max = 10 # maximum value

# Initialise the arrays with zeros
x1_values = zeros(N)
x2_values = zeros(N)
u_values = zeros(N, N);

# fill the arrays with the appropriate values
for i in 1:N
    for j in 1:N
        x1_values[i] = (i / N) * x_max 
        x2_values[j] = (j / N) * x_max
        u_values[i, j] = u_func(x1_values[i], x2_values[j])
    end
end

**Note** In order to understand this loop you need to take things slowly the first time. Try and understand what is happening in each step. I have provided a detailed description below, but you still need to sit and think through it. 

Let us analyse the loop above and see what is happening at each step. First, this is a nested loop. As we stated in our previous session, this means that we will be running through the $i$ component first and then the corresponding $j$ component. 

To use an example, we start with $i = 1$, and then run the loop over all the $j$ values starting at $j = 1$ and ending at $j = N = 100$. 

This means that for $i = 1$, we start with $j = 1$. This gives a value for `x1_values[1] = (1 / N) * x_max`. 

This means that the first value in the `x1_values` vector will then be $(1 / 100) * 10 = 0.1$. The same is true for `x2_values`, the first entry in that vector will be $0.1$, by the same logic as applied before. 

Now we can enter these $x_1$ and $x_2$ values into the utility function. The shape of the utility function as initialised is a $N \times N$ matrix of zeros. We are currently busy with $i = 1$ and $j = 1$. This means that in our code we are looking at `u_values[1, 1]` which is the value in the first column and first row of the matrix. 

To be clear, `u_values[1, 1]` takes on the value from the output of `u_func(0.1, 0.1)`. 

Next we keep the $i$ component fixed at $1$ and iterate over the next value in the $j$ sequence, which is $j = 2$. We repeat the steps as for the above description and end up with an entry for `u_values[1, 2]`. 

Keeping $i$ fixed, we iterate over all the values for $j$. That means for $j = 3$ to $j = 100$. After having completed this loop, we move on to a new value for $i$. 

Move on to $i = 2$ and repeat the entire process that we described above again. Looping over all the $j$ values, etc. After having done that move on to $i = 3$ and then continue the process we have described till you reach $i = 100$.

At the end of all of this you will see that we have know completely filled the `u_values` matrix with all possible combinations of $x_1$ and $x_2$ values (smaller than $x_{max} = 10$) that can enter the utility function. 